# Women In Data Science - Kaggle Competition 


## User ID: RinzlerTron ; Team: Songpyeon
## Rank - #27 on Public Leaderboard and Rank #30 on Private Leaderboard

https://www.kaggle.com/c/wids2018datathon

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
#from fastai.imports import *
#from fastai.structured import *
from pandas_summary import DataFrameSummary
%matplotlib inline 
import missingno as mn
from sklearn.linear_model import LogisticRegression
from catboost import *
from fancyimpute import  KNN, NuclearNormMinimization, SoftImpute
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

Using TensorFlow backend.
E:\Other\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [90]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
del train['train_id']
id = test['test_id']
del test['test_id']
train.shape, test.shape

((18255, 1234), (27285, 1233))

In [91]:
col = train.isnull().sum().sort_values(ascending = False)<0.99*train.shape[0]
train = train.loc[:,col]
train.head(3)

,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,DG3,...,LN2_2,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5
0,3,32,3.0,NaN,323011,3854,481,1975,1,3,...,1,1,1,NaN,NaN,99.0,99,99,99,99
1,2,26,NaN,8.0,268131,2441,344,1981,1,8,...,1,3,4,Bengali,Bengali,NaN,1,2,2,2
2,1,16,NaN,7.0,167581,754,143,1995,1,3,...,1,2,2,Hindi,Hindi,1.0,2,2,2,2


In [92]:
index = col[(col == True)].index
test = test.loc[:,index]
del test['is_female']
test.shape, train.shape

((27285, 699), (18255, 700))

### Convert all float into int

In [93]:
col2 = list(train.select_dtypes(include=['object']).columns)
col2

['DL1_OTHERS',
 'DL2_96_OTHERS',
 'DL4_OTHERS',
 'MM12_REC',
 'FL9A_OTHERS',
 'FB19_6_OTHERS',
 'LN2_RIndLngBEOth',
 'LN2_WIndLngBEOth']

In [94]:
train.loc[:,col2] = train.loc[:,col2].fillna("missing")
test.loc[:,col2] = test.loc[:,col2].fillna("missing")

In [95]:
train["row_nulls"] = train.isnull().sum(axis=1)/float(train.shape[1])
test["row_nulls"] = test.isnull().sum(axis=1)/float(test.shape[1])
train.fillna(1000,inplace=True)
test.fillna(1000,inplace=True)

In [96]:
float_col = list(train.select_dtypes(include=['float64']).columns)

In [97]:
train[float_col] = train[float_col].astype("int64")
test[float_col] = test[float_col].astype("int64")

In [98]:
train.head(3)

,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,DG3,...,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5,row_nulls
0,3,32,3,1000,323011,3854,481,1975,1,3,...,1,1,missing,missing,99,99,99,99,99,0
1,2,26,1000,8,268131,2441,344,1981,1,8,...,3,4,Bengali,Bengali,1000,1,2,2,2,0
2,1,16,1000,7,167581,754,143,1995,1,3,...,2,2,Hindi,Hindi,1,2,2,2,2,0


In [99]:
col = list(train.select_dtypes(include=['object','int64']).columns)
v = (train[col].apply(lambda x: x.nunique(), axis=0)>3).reset_index()
col_filtered = list(v.loc[(v.iloc[:,1]==True),'index'])
u = (train[col_filtered].nunique()<11).reset_index()
col_filtered = list(u.loc[(u.iloc[:,1]==True),'index'])

### Target mean encoding on col_filtered 

In [100]:
def mean_encoding(df, test, col):
    for item in col:
        mean_device_type = df.groupby(item).is_female.mean()
        test[item+"_mean_enc"] = test[item].map(mean_device_type)
        df[item+"_mean_enc"] = df[item].map(mean_device_type)
    return df,test

In [101]:
def process_data(train,test):
    combine = pd.concat([train,test])
    object_cols = list(test.select_dtypes(include=['object','category']).columns)
    for c in object_cols:
        combine[c] = combine[c].astype('category').cat.codes+1  
    train = combine.iloc[:train.shape[0],]
    test = combine.iloc[train.shape[0]:,]
    return train,test

In [102]:
train,test = mean_encoding(train, test, col_filtered)
train,test = process_data(train,test)

In [103]:
global_mean = train.is_female.mean()
test.fillna(global_mean, inplace=True)

In [104]:
del test['is_female']
test.head(3), train.shape

(   AA14  AA15  AA3  AA3_mean_enc  AA4   AA5  AA5_mean_enc   AA6  AA6_mean_enc  \
 0  4479   535    4      0.541462   41  1000      0.538803     7      0.534333   
 1  3803   476    3      0.476144   32     2      0.524664  1000      0.533345   
 2  5610   585    3      0.476144   36     5      0.528686  1000      0.533345   
 
       AA7    ...      MT6B  MT6B_mean_enc  MT6C  MT6_mean_enc   MT7  MT7A  \
 0  417211    ...      1000       0.701275  1000      0.701275     2  1000   
 1  322011    ...         1       0.363453     0      0.179292  1000  1000   
 2  365011    ...      1000       0.701275  1000      0.701275     2  1000   
 
    MT7A_mean_enc   MT8   MT9  row_nulls  
 0       0.488904  1000    12          0  
 1       0.488904  1000  1000          0  
 2       0.488904  1000    13          0  
 
 [3 rows x 888 columns], (18255, 889))

In [105]:
# mn.bar(test)

In [106]:
# train_filled_knn = KNN(k=3).complete(train)
# train_filled_knn = pd.DataFrame(train_filled_knn, columns=train.columns, index = train.index)
# train_filled_knn.shape

In [107]:
# test_filled_knn = KNN(k=3).complete(test)
# test_filled_knn = pd.DataFrame(test_filled_knn, columns=test.columns, index = test.index)
# test_filled_knn.shape

In [2]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("train_afterbin.csv")
test = pd.read_csv("test_afterbin.csv")

In [4]:
y = train['is_female']
X = train.drop(['is_female'], axis = 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=10)

In [110]:
def auc(m, train, val): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_val,m.predict_proba(val)[:,1]))

### Parameter Tuning Random Search XGBoost

In [112]:
# model = xgb.XGBClassifier()
# param_dist = {"max_depth": [15, 20, 25,30],
#               "min_child_weight" : [3,5,10]}

# # run randomized search
# n_iter_search = 10
# random_search = RandomizedSearchCV(model, param_distributions=param_dist, cv = 4, 
#                                    n_iter=n_iter_search,verbose=1, n_jobs=-1)
# random_search.fit(X_train, y_train)
# random_search.best_estimator_

### Fitting Final Model XGBoost

In [ ]:
#model = xgb.XGBClassifier(max_depth=20, min_child_weight=8,  n_estimators=100, n_jobs=-1 )



#model.fit(X_train,y_train)

#auc(model, X_train, X_val)

In [5]:
#model = xgb.XGBClassifier(max_depth=20, min_child_weight=8,  n_estimators=100)
model = xgb.XGBClassifier(max_depth=6, min_child_weight=1.36,  n_estimators=540, colsample_bytree= 0.6, gamma= 0.05, subsample=1, learning_rate=0.05)
#model.fit(X_train,y_train)
#colsample_bytree
#Depth: 6 Eta: 0.05 Subsample: 1 Colsample: 0.6 Gamma: 0.05 Minchild weight: 1.36 Alpha: 0"
#auc(model, X_train, X_val)

#(0.9990936995241299, 0.9711375605506248)
#model = xgb.XGBClassifier(max_depth=6, min_child_weight=1.36,  n_estimators=540, colsample_bytree= 0.6, gamma= 0.05, subsample=1, learning_rate=0.05)

In [6]:
model.fit(X,y)
pred = model.predict_proba(test)

In [7]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_xgboost.csv", index=False)

In [114]:
# test.loc[:,'is_female'] = pred[:,1]
# test.loc[(test['DL2_96_OTHERS']=="House wife"),'is_female']
# test['DL2_96_OTHERS']=="House wife"
# (train[col_filtered].nunique()>10).reset_index()

### Catboost Classifier - Base model

In [64]:
cate_features_index = np.where(X.dtypes != float)[0]

In [116]:
# train default catBoost classifier. Default loss metric is LogLoss (lower is better)
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=6, iterations=400)

In [ ]:
# clf.fit(X,Y , cat_features=cat_dims)
clf.fit(X_train,y_train, cat_features=cate_features_index)

In [86]:
auc(clf, X_train, X_val)

(0.9753617349262516, 0.9690407385145595)

In [ ]:
clf.fit(X,y, cat_features=cate_features_index)

In [118]:
pred = clf.predict_proba(test)

In [119]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_base_Cat.csv", index=False)

### Parameter Tuning

In [164]:
from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product,chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb

''' a class for doing grid search on a set of parameters provided in a dict. 'pdict' should be a dictionary like the following:
pdict = {'depth':[1,2], 'iterations':[250,100,500], 'thread_count':4}

when grid_search is called it will return an iterator that provides samples from the dictionary e.g.
{'depth':1, 'iterations':250, 'thread_count':4}
{'depth':2, 'iterations':250, 'thread_count':4}
{'depth':1, 'iterations':100, 'thread_count':4}
etc.
after calling an iteration of grid_search, you need to test the classifier and run 'register_result'
This will update the internal list of results, so that the next call to grid_search will use the best
parameters for all the parameters not currently being updated.

grid_search can be provided a list e.g. grid_search(['depth']) this will use the current best parameters for all
the other arguments and only search over 'depth'. You can then call e.g. grid_search(['iterations']) and it will use
the best depth found previously and cycle through all the 'iterations'. Searching incrementally can be much faster
than doing a full grid search, but may miss the global optimum. '''
class paramsearch:
    def __init__(self,pdict):    
        self.pdict = {}
        # if something is not passed in as a sequence, make it a sequence with 1 element
        #   don't treat strings as sequences
        for a,b in pdict.items():
            if isinstance(b, collections.Sequence) and not isinstance(b, str): self.pdict[a] = b
            else: self.pdict[a] = [b]
        # our results are a sorted list, so the best score is always the final element
        self.results = SortedList()       
                    
    def grid_search(self,keys=None):
        # do grid search on only the keys listed. If none provided, do all
        if keys==None: keylist = self.pdict.keys()
        else: keylist = keys
 
        listoflists = [] # this will be list of lists of key,value pairs
        for key in keylist: listoflists.append([(key,i) for i in self.pdict[key]])
        for p in product(*listoflists):
            # do any changes to the current best parameter set
            if len(self.results)>0: template = self.results[-1][1]
            else: template = {a:b[0] for a,b in self.pdict.items()}
            # if our updates are the same as current best, don't bother
            if self.equaldict(dict(p),template): continue
            # take the current best and update just the ones to change
            yield self.overwritedict(dict(p),template)
                              
    def equaldict(self,a,b):
        for key in a.keys(): 
            if a[key] != b[key]: return False
        return True            
                              
    def overwritedict(self,new,old):
        old = copy.deepcopy(old)
        for key in new.keys(): old[key] = new[key]
        return old            
    
    # save a (score,params) pair to results. Since 'results' is a sorted list,
    #   the best score is always the final element. A small amount of noise is added
    #   because sorted lists don't like it when two scores are exactly the same    
    def register_result(self,result,params):
        self.results.add((result+np.random.randn()*1e-10,params))    
        
    def bestscore(self):
        return self.results[-1][0]
        
    def bestparam(self):
        return self.results[-1][1]

In [165]:
params = {'depth':[2,5,10],
          'iterations':[100],
          'learning_rate':[0.03,0.6,0.3], 
          'l2_leaf_reg':[1,5,30]}

In [166]:
# this function does 3-fold crossvalidation with catboostclassifier          
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)

In [ ]:
# this function runs grid search on several parameters
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams = catboost_param_tune(params,X_train,y_train,cat_dims =cate_features_index)

In [169]:
bestparams

{'depth': 10, 'iterations': 100, 'l2_leaf_reg': 5, 'learning_rate': 0.6}

In [ ]:
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=11, \
                        depth=10, iterations= 300, l2_leaf_reg= 5, learning_rate= 0.08)
clf.fit(X_train,y_train, cat_features=cate_features_index)
auc(clf, X_train, X_val)

In [ ]:
clf.fit(X,y, cat_features=cate_features_index)
pred = clf.predict_proba(test)

In [176]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_tuned_Cat.csv", index=False)

In [177]:
pred_mean = (pred[:,1] + pd.read_csv("Submission_xgboost.csv")['is_female'])/2

In [183]:
submit['is_female'] = pred_mean
submit.to_csv("FirstEnsemble.csv", index=False)

In [184]:
pred_mean3 = (pred[:,1] + pd.read_csv("Submission_xgboost.csv")['is_female'] +\
             pd.read_csv("Submission_base_Cat.csv")['is_female'])/3
submit['is_female'] = pred_mean3
submit.to_csv("SecondEnsemble.csv", index=False)

### Random Forest

In [55]:
m = RandomForestClassifier(n_estimators=200, max_features= 100,min_samples_leaf=8, max_depth=80)
m.fit(X_train, y_train)
auc(m, X_train, X_val)

(0.9899227020756343, 0.9648185150889961)

In [ ]:
m.fit(X,y)
pred = m.predict_proba(test)

In [ ]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_RF.csv", index=False)

### Logistic Regression after Normalizing

In [21]:
# scaler = StandardScaler()
# scaler.fit(train_filled_knn.drop(['is_female'], axis =1))
# scaled_train = scaler.transform(train_filled_knn.drop(['is_female'], axis =1))
# scaled_test = scaler.transform(test_filled_knn)

# scaled_train = pd.DataFrame(scaled_train, columns=train_filled_knn.drop(['is_female'], axis =1).columns, index=train_filled_knn.drop(['is_female'], axis =1).index)
# scaled_test = pd.DataFrame(scaled_test, columns=test_filled_knn.columns, index=test_filled_knn.index)

# scaled_train["is_female"] =  train_filled_knn["is_female"]
# x_train ,x_val = train_test_split(scaled_train,test_size=0.2) 

# y_train = x_train["is_female"]
# del x_train["is_female"]
# y_valid = x_val['is_female']
# del x_val['is_female']

# def auclr(lr, train, val): 
#     return (metrics.roc_auc_score(y_train,lr.predict_proba(train)[:,1]),
#                             metrics.roc_auc_score(y_valid,lr.predict_proba(val)[:,1]))
# lr = LogisticRegression()
# lr.fit(x_train, y_train)
# auclr(lr, x_train, x_val)

# y_train = scaled_train["is_female"]
# x_train = scaled_train.copy()
# del x_train["is_female"]
# lr.fit(x_train, y_train)

# submit = pd.DataFrame()
# submit['test_id'] = id
# submit['is_female'] = lr.predict_proba(scaled_test)[:,1]
# submit.to_csv("Submission_LR.csv", index=False)